## Data Pre-processing

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
df = pd.read_csv("weatherAUS.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(labels = ['Date','Location','Evaporation','Sunshine','Cloud3pm','Cloud9am','RISK_MM'],axis = 1,inplace = True)

In [ ]:
# Replace Nilai Yes / No
df['RainToday'].replace({'No':0,'Yes':1},inplace = True) 
df['RainTomorrow'].replace({'No':0,'Yes':1},inplace = True)

In [ ]:
#Remove missing values
df.dropna(inplace = True) 

In [ ]:
categorical = ['WindGustDir','WindDir9am','WindDir3pm']
df_final = pd.get_dummies(df,columns = categorical,drop_first=True)

In [ ]:
df_final.head()

## Separating Your Training and Testing Datasets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_final.drop(['RainTomorrow'],axis=1).values
y = df_final['RainTomorrow'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Transforming the Data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Artificial Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Dense(6, kernel_initializer = "uniform",activation = "relu", input_dim=58))
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [ ]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [ ]:
classifier.fit(X_train, y_train, batch_size = 500, epochs = 25)

## Running Predictions on the Test Set

In [ ]:
y_pred = classifier.predict(X_test)
y_train_pred = classifier.predict_classes(X_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('Training Accuracy ---->',accuracy_score(y_train,y_train_pred))
print('Testing Accuracy  ---->',accuracy_score(y_test,y_pred.round()))